In [223]:
import argparse
import os
import numpy as np
import math
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
from torchvision.utils import save_image
import torchvision.datasets as dset

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [224]:
n_epochs = 100
batch_size = 64
lr = 1e-4
b1 = 0.5
b2 = 0.999
n_cpu = 8
latent_dim = 100
img_size = 64
channels = 3
sample_interval=400
dataroot = "resized_data"

img_shape = (channels, img_size, img_size)

cuda = True if torch.cuda.is_available() else False

In [225]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img

In [226]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)

        return validity

In [227]:
# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

In [228]:
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(img_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

In [229]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True)

In [230]:
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [232]:
for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        # Adversarial ground truths
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 3, (imgs.shape[0], latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()
        
        batches_done = epoch * len(dataloader) + i

        if batches_done % sample_interval == 0:
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
            )
            save_image(gen_imgs.data[:25], "gen_images_bayc_no_background/%d.png" % batches_done, nrow=5, normalize=True)    

[Epoch 0/100] [Batch 0/157] [D loss: 0.439645] [G loss: 0.681180]
[Epoch 2/100] [Batch 86/157] [D loss: 0.537516] [G loss: 0.757705]
[Epoch 5/100] [Batch 15/157] [D loss: 0.471407] [G loss: 0.669744]
[Epoch 7/100] [Batch 101/157] [D loss: 0.562650] [G loss: 0.634921]
[Epoch 10/100] [Batch 30/157] [D loss: 0.497849] [G loss: 0.920775]
[Epoch 12/100] [Batch 116/157] [D loss: 0.603179] [G loss: 0.674976]
[Epoch 15/100] [Batch 45/157] [D loss: 0.521645] [G loss: 1.090696]
[Epoch 17/100] [Batch 131/157] [D loss: 0.456159] [G loss: 0.964809]
[Epoch 20/100] [Batch 60/157] [D loss: 0.781148] [G loss: 2.443239]
[Epoch 22/100] [Batch 146/157] [D loss: 0.430493] [G loss: 0.952679]
[Epoch 25/100] [Batch 75/157] [D loss: 0.469555] [G loss: 0.765875]
[Epoch 28/100] [Batch 4/157] [D loss: 0.425406] [G loss: 0.973689]
[Epoch 30/100] [Batch 90/157] [D loss: 0.460470] [G loss: 1.362535]
[Epoch 33/100] [Batch 19/157] [D loss: 0.467158] [G loss: 1.227550]
[Epoch 35/100] [Batch 105/157] [D loss: 0.548767] 